In [1]:
import os
import json
import pprint
import re
import pkg_resources
import pandas as pd
import optparse

pp = pprint.PrettyPrinter(indent=4)

def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


In [172]:
def load_processdict(acquisition_dir, run):
    processdict = None
    #processdict_table = None 
    processdict_filepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.json' % run)
    #processdict_tablepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.txt' % run)
    
    # Load analysis "processdict" file:
    if os.path.exists(processdict_filepath):
        print "exists!"
        with open(processdict_filepath, 'r') as f:
            processdict = json.load(f)

#     # Also load TABLE version: 
#     if os.path.exists(processdict_tablepath):        
#         processdict_table = pd.read_csv(processdict_tablepath, sep='\s+', header=0, index_col=0)
    
    return processdict #, processdict_table
              


In [148]:
print acquisition_dir
#processdict, processdict_table = load_processdict(acquisition_dir, run)
processdict = load_processdict(acquisition_dir, run)

/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x
/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/processed/pid_info_gratings_phasemod_run1.json
exists!


In [149]:
processdict

{u'processed001': {u'DST': u'/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/processed/processed001',
  u'PARAMS': {u'motion': {u'algorithm': None,
    u'correct_motion': False,
    u'method': None,
    u'ref_channel': None,
    u'ref_file': None},
   u'preprocessing': {u'bidir_corrected': False,
    u'flyback_corrected': False,
    u'nflyback_frames': None},
   u'source': {u'acquisition': u'FOV1_zoom3x',
    u'animalid': u'CE059',
    u'rootdir': u'/nas/volume1/2photon/data',
    u'run': u'gratings_phasemod_run1',
    u'session': u'20171009_CE059'}},
  u'SRC': u'/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/raw',
  u'process_id': u'processed001',
  u'version': u'0.1.0'}}

In [166]:
def get_process_id(processdict):
    
    print "************************"
    print "Generating PROCESS ID..."
    print "************************"
    
    process_id = None
    
    # First check current params against existing analyses:
    if processdict is None or len(processdict.keys()) == 0:
        processdict = dict()
        existing_pids = []
        is_new_pid = True
        print "No existing PIDs found."
        create_pid_file = True
    else:
        create_pid_file = False
        existing_pids = sorted([str(k) for k in processdict.keys()], key=natural_keys)

        # Show existing PIDs:
        while True:
            print "Found existing PIDs:"
            for pidx, pid in enumerate(existing_pids):
                print pidx, pid
            
            check_pidx = raw_input('Enter IDX of pid to view or hit <ENTER> to create new: ')
            if len(check_pidx) == 0:
                is_new_pid = True
                break
            else:
                print "Viewing PID: %s" % existing_pids[int(check_pidx)]
                pp.pprint(processdict[existing_pids[int(check_pidx)]])
                reuse_idx = raw_input('Press <R> to re-use current pid, or hit <ENTER> to view list.')
                if reuse_idx == 'R':
                    is_new_pid = False
                    break
 
    if is_new_pid is True:
        # Create new PID by incrementing num of process dirs found:
        process_id = 'processed%03d' % int(len(existing_pids)+1)
        print "Creating NEW pid: %s" % process_id
    else:
        # Re-using an existing PID:
        process_id = existing_pids[int(check_pidx)]
        print "Reusing existing pid: %s" % process_id
 
    return process_id, is_new_pid, create_pid_file #, processdict

In [113]:
def get_basic_pid(rootdir=rootdir, animalid=animalid, session=session,
                         acquisition=acquisition, run=run):
    tiffsource = 'raw'
    correct_motion = False
    correct_bidir = False
    
    pid = set_processing_params(rootdir=rootdir, animalid=animalid, session=session,
                         acquisition=acquisition, run=run, tiffsource=tiffsource,
                         correct_motion=correct_motion, correct_bidir=correct_bidir, correct_flyback=correct_flyback)
    
    return pid

In [144]:
basic_pid = get_basic_pid(rootdir=rootdir, animalid=animalid, session=session,
                         acquisition=acquisition, run=run)
pp.pprint(basic_pid)

if processdict is None:
    processdict = dict()
    processdict[basic_pid['process_id']] = basic_pid
    pp.pprint(processdict)

pidinfo_basename = 'pid_info_%s' % run
print pidinfo_basename

with open(os.path.join(acquisition_dir, run, 'processed', '%s.json' % pidinfo_basename), 'w') as f:
    json.dump(processdict, f, indent=4, sort_keys=True)
    

{   'algorithm': None,
    'correct_motion': False,
    'method': None,
    'ref_channel': None,
    'ref_file': None}
/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/processed/pid_info_gratings_phasemod_run1.json
None
No existing PIDs found.
Creating NEW pid: processed001
{   'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/processed/processed001',
    'PARAMS': {   'motion': {   'algorithm': None,
                                'correct_motion': False,
                                'method': None,
                                'ref_channel': None,
                                'ref_file': None},
                  'preprocessing': {   'bidir_corrected': False,
                                       'flyback_corrected': False,
                                       'nflyback_frames': None},
                  'source': {   'acquisition': 'FOV1_zoom3x',
                                'animalid': 'CE059',
 

In [167]:
def initialize_pid(PARAMS, process_dict, acquisition_dir, run, tiffsource):
    
    print "************************"
    print "Initialize PID."
    print "************************"

    pid = dict()
    version = pkg_resources.get_distribution('pipeline').version
    pid['version'] = version 
    
    processed_dirs = sorted([p for p in os.listdir(os.path.join(acquisition_dir, run, 'processed'))
                              if 'processed' in p], key=natural_keys)
    
    if tiffsource is None:
        while True:
            print "TIFF SOURCE was not specified."
            tiffsource_type = raw_input('Enter <P> if source is processed, <R> if raw: ')
            if tiffsource_type == 'R':
                tiffsource = 'raw'
                break
            elif tiffsource_type =='P':
                print "Selected PROCESSED source."
                pp.pprint(processed_dirs)
                if process_dict is None or (len(process_dict.keys()) == 0 and len(processed_dirs) == 0):
                    tiffsource = 'processed%03d' % int(len(processed_dirs) + 1)
                else:
                    candidate_sources = sorted(process_dict.keys(), key=natural_keys)
                    for pdix, pdir in enumerate(candidate_sources):
                        print pdix, pdir
                    tiffsource_idx = input('Select IDX of processed dir to use as source: ')
                    if tiffsource_idx < len(candidate_sources):
                        tiffsource = candidate_sources[int(tiffsource_idx)]
                confirm_selection = raw_input('Selected %s. Press <Y> to confirm.' % tiffsource)
                if confirm_selection == 'Y':
                    break

    # SET SOURCE:
    if tiffsource is 'raw':
        pid['SRC'] = os.path.join(acquisition_dir, run, tiffsource) #source_dir
    else:
        pid['SRC'] = os.path.join(acquisition_dir, run, 'processed', tiffsource) 
        
    # SET DEST:

    process_id, is_new_pid, create_pid_file = get_process_id(process_dict)
    
    pid['DST'] = os.path.join(acquisition_dir, run, 'processed', process_id)
    
    pid['process_id'] = process_id

    pid['PARAMS'] = PARAMS
 
    return pid
     


In [83]:
def set_mcparams(acquisition_dir, run,
                    correct_motion=True,
                    ref_channel=1,
                    ref_file=1,
                    method=None,
                    algorithm=None,
                    ):

    mcparams = dict()
    mcparams['correct_motion'] = correct_motion
    mcparams['method'] = method
    mcparams['algorithm'] = algorithm
    mcparams['ref_channel'] = ref_channel
    mcparams['ref_file'] = ref_file
#     mcparams['source_dir'] = source
#     mcparams['dest_dir'] = destination

#     if source is None:
#         mcparams['source_dir'] = os.path.join(acquisition_dir, run, 'raw')
#     if destination is None:
#         mcparams['dest_dir'] = os.path.join(acquisition_dir, run, 'processed', process_id)

    return mcparams


In [9]:
def check_process_id(tmp_pid, processdict):

    existing_pids = processdict.keys()
    
    # Check if all PID fields match existing ones...
#     keys_to_check = 

    matching_analysis = sorted([epid for epid in existing_pids if 
                                len([i for i in tmp_pid.keys() if tmp_pid[i] == processdict[epid][i]]) == len(tmp_pid.keys())], key=natural_keys)
    if len(matching_analysis) > 0:    
        for m, mi in enumerate(matching_analysis):
            print m, mi
            
        while True:
            user_choice = raw_input("Found matching analysis ID. Press N to create new, or IDX of analysis_id to reuse: ")
            
            if user_choice == 'N':
                is_new_pid = True
                process_id = "analysis%02d" % int(len(existing_pids)+1)
            elif user_choice.isdigit():
                is_new_pid = False
                process_id = matching_analysis[int(user_choice)]

            print "Selected processing params:"
            pp.pprint(processdict[process_id]) 

            confirm = raw_input('Using analysis ID: %s. Press Y/n to confirm: ' % process_id)
            if confirm == 'Y':
                break
            
    else:
        print "New process params specified. Creating new pid."
        is_new_pid = True
                
    if is_new_pid is True:
        # Create new PID by incrementing num of process dirs found:
        process_id = 'process%03d' % int(len(existing_pids)+1)
        print "Creating NEW pid: %s" % process_id
       
    else:
        # Check re-used analysis fields and make sure they all match:
        for field in processdict[process_id].keys():
            if field == 'process_id':
                continue
            if not processdict[process_id][field] == I[field]:
                # Stay in loop until user supplies valid entries:
                while True:
                    print "Mismatch found in field: %s" % field
                    print "[0] OLD: %s || [1] NEW: %s" % (processdict[process_id][field], I[field])
                    overwrite_choice = input('Press 0 to keep old, or 1 to overwrite with new value: ')
                    if int(overwrite_choice) == 0:
                        print "Selected to keep OLD key-value pair:", field, processdict[process_id][field]
                        confirm_choice = input('Press Y to confirm: ')
                    elif int(overwrite_choice) == 1:
                        print "Selected to overwrite with NEW key-value pair:", field, I[field]
                        confirm_choice = input('Press Y to confirm: ')
                    
                    if confirm_choice == 'Y' and int(overwrite_choice) == 0: 
                        I[field] = processdict[process_id][field]
                        break
                    elif confirm_choice == 'Y' and int(overwrite_choice) == 1: 
                        processdict[process_id][field] = I[field]
                        break

In [173]:
#def update_records(pid, processdict, processdict_table, acquisition_dir, run):
def update_records(pid, processdict, acquisition_dir, run):

    print "************************"
    print "Updating JSONS..."
    print "************************"

    processdict_filepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.json' % run)
    #processdict_tablepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.txt' % run)

    if processdict is None:
        processdict = dict()
    process_id = pid['process_id'] 
    processdict[process_id] = pid
        
#     new_table_entry = pd.DataFrame.from_dict({process_id: pid}, orient='index')
#     if processdict_table is None:
#         processdict_table = new_table_entry
#     else:
#         processdict_table.append(new_table_entry)
    
    #% Update Process Info DICT:
    with open(processdict_filepath, 'w') as f:
        json.dump(processdict, f, sort_keys=True, indent=4)
       
    #% Update Process Info TABLE: 
    #processdict_table.to_csv(processdict_tablepath, sep='\t', header=True, index=True, index_label='Row') #header=False)

#    with open(acquisition_meta_fn, 'w') as f:
#        json.dump(acqmeta, f, sort_keys=True, indent=4)
#    
    #scipy.io.savemat(acquisition_meta_mat, mdict=acqmeta)
    
    print "Process Info UPDATED."


In [174]:
# Create PARAMS file:
# This should be main()?
def set_processing_params(rootdir=rootdir, animalid=animalid, session=session,
                         acquisition=acquisition, run=run, tiffsource=None,
                         correct_bidir=False, correct_flyback=False, nflyback_frames=None,
                         correct_motion=False, ref_file=1, ref_channel=1,
                         mc_method=None, mc_algorithm=None):
    
    mc_methods = ['Acquisition2P', 'NoRMCorre']
    mc_algos = dict((mc, []) for mc in mc_methods)
    mc_algos = {'Acquisition2P': ['@withinFile_withinFrame_lucasKanade', '@lucasKanade_plus_nonrigid'],
                'NoRMCorre': ['rigid', 'nonrigid']}

    PARAMS = dict()
    PARAMS['source'] = dict()
    PARAMS['preprocessing'] = dict()
    PARAMS['motion'] = dict()

    acquisition_dir = os.path.join(rootdir, animalid, session, acquisition)
    
    # ----------------------------------------------------------
    # Get preprocessing opts:
    # ----------------------------------------------------------
    PARAMS['source']['rootdir'] = rootdir
    PARAMS['source']['animalid'] = animalid
    PARAMS['source']['session'] = session
    PARAMS['source']['acquisition'] = acquisition
    PARAMS['source']['run'] = run

    PARAMS['preprocessing']['flyback_corrected'] = correct_flyback
    PARAMS['preprocessing']['nflyback_frames'] = nflyback_frames
    PARAMS['preprocessing']['bidir_corrected'] = correct_bidir


    # ------------------------------------------
    # Check user-provided MC params:
    # ------------------------------------------
    if correct_motion is True:
        if mc_method is None:
            while True:
                print "No MC method specified. Use default [Acquisition2P]?"
                mc_choice = raw_input('Enter <Y> to use default, or <o> to see options:')
                if mc_choice == 'Y':
                    print "Using default."
                    mc_method = 'Acquisition2P'
                    break
                elif mc_choice == 'o':
                    for mcid, mcname in enumerate(mc_methods):
                        print mcid, mcname
                    mc_select = input('Enter IDX of motion-correction method to use:')
                    mc_method = mc_methods[mc_select]
                    break

        if mc_algorithm is None or (mc_algorithm not in mc_algos[mc_method]):
            print "No MC algorithm specified... Here are the options:"
            for algonum, algoname in enumerate(mc_algos[mc_method]):
                print algonum, algoname
            algo_choice = input('Enter IDX of mc algorithm to use:')
            mc_algorithm = mc_algos[mc_method][algo_choice] 
    else:
        ref_channel = None
        ref_file = None
        mc_method = None
        mc_algorithm = None

    # Set MCPARAMS:
    mcparams = set_mcparams(acquisition_dir, run,
                            correct_motion=correct_motion,
                            ref_channel=ref_channel,
                            ref_file=ref_file,
                            method=mc_method,
                            algorithm=mc_algorithm)
    pp.pprint(mcparams)

    PARAMS['motion'] = mcparams

    # INITIALIZE PID:
#     processdict, processdict_table = load_processdict(acquisition_dir, run)
    processdict = load_processdict(acquisition_dir, run)

    pp.pprint(processdict)
    pid = initialize_pid(PARAMS, processdict, acquisition_dir, run, tiffsource)
    
    # UPDATE RECORDS:
#     update_records(pid, processdict, processdict_table, acquisition_dir, run)
    update_records(pid, processdict, acquisition_dir, run)
    
    # STORE TMP FILE OF CURRENT PARAMS:
    with open(os.path.join(acquisition_dir, run, 'processed', 'tmp_processparams.json'), 'w') as f:
        json.dump(pid, f, indent=4, sort_keys=True)
        
    return pid

In [175]:
# -----------------------------------------------
# Command-line opts:
# -----------------------------------------------
#parser = optparse.OptionParser()

# parser.add_option stuff goes here

#(options, args) = parser.parse_args(options)

rootdir = '/nas/volume1/2photon/data' #options.root
animalid = 'CE059' #options.animalid
session = '20171009_CE059' #options.session
acquisition = 'FOV1_zoom3x' #options.acquisition
run = 'gratings_phasemod_run1' #options.run

tiffsource = None #'processed001' #options.source

correct_motion = True #options.mc
correct_bidir = True #options.bidi
correct_flyback = False
nflyback_frames = None

ref_file = 1 #options.ref_file
ref_channel = 1 #options.ref_channel
mc_method = None #options.mc_method
mc_algorithm = None #options.algorithm

In [176]:
pid = set_processing_params(rootdir=rootdir, animalid=animalid, session=session,
                           acquisition=acquisition, run=run, tiffsource=tiffsource,
                           correct_bidir=correct_bidir, correct_flyback=correct_flyback, nflyback_frames=nflyback_frames,
                         correct_motion=correct_motion, ref_file=ref_file, ref_channel=ref_channel,
                         mc_method=mc_method, mc_algorithm=mc_algorithm)
pp.pprint(pid)

No MC method specified. Use default [Acquisition2P]?
Enter <Y> to use default, or <o> to see options:Y
Using default.
No MC algorithm specified... Here are the options:
0 @withinFile_withinFrame_lucasKanade
1 @lucasKanade_plus_nonrigid
Enter IDX of mc algorithm to use:0
{   'algorithm': '@withinFile_withinFrame_lucasKanade',
    'correct_motion': True,
    'method': 'Acquisition2P',
    'ref_channel': 1,
    'ref_file': 1}
exists!
{   u'processed001': {   u'DST': u'/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_phasemod_run1/processed/processed001',
                         u'PARAMS': {   u'motion': {   u'algorithm': None,
                                                       u'correct_motion': False,
                                                       u'method': None,
                                                       u'ref_channel': None,
                                                       u'ref_file': None},
                                        u'p